In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
%fs ls dbfs:/FileStore/tables/


path,name,size,modificationTime
dbfs:/FileStore/tables/csv/,csv/,0,0
dbfs:/FileStore/tables/json/,json/,0,0
dbfs:/FileStore/tables/parquet/,parquet/,0,0
dbfs:/FileStore/tables/train/,train/,0,0


In [0]:
df_csv = spark.read.format("csv").option("header",True).load("dbfs:/FileStore/tables/csv/batch.csv")

In [0]:
type(df_csv)

Out[90]: pyspark.sql.dataframe.DataFrame

In [0]:
#df_csv.show()
df_csv.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- department: string (nullable = true)



In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType, StringType,DateType

In [0]:
sch = StructType([
    StructField("id",IntegerType()),
    StructField("name",StringType()),
    StructField("dob",DateType()),
    StructField("age",IntegerType()),
    StructField("salary",IntegerType()),
    StructField("department",StringType()),
])

In [0]:
df_csv = spark.read.format("csv").schema(sch).option("header",True).load("dbfs:/FileStore/tables/csv/batch.csv")

In [0]:
df_csv.show()

+---+-----+----------+----+------+----------+
| id| name|       dob| age|salary|department|
+---+-----+----------+----+------+----------+
|  1| John|1992-05-12|  30| 70000|        IT|
|  2|Alice|1997-02-28|  25| 60000|        HR|
|  3|  Bob|      null|null| 80000|        IT|
|  4|Emily|1994-11-22|  28| 65000|   Finance|
+---+-----+----------+----+------+----------+



In [0]:
df_csv.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department: string (nullable = true)



In [0]:
df_json = spark.read.format("json").load("dbfs:/FileStore/tables/json")

In [0]:
df_json.show()

+----+----------+----------+---+------+------+
| age|department|       dob| id|  name|salary|
+----+----------+----------+---+------+------+
|  30|        IT|1992-05-12|  1|  John| 70000|
|  25|        HR|1997-02-28|  2| Alice| 60000|
|null|        IT|      null|  3|   Bob| 80000|
|  28|   Finance|1994-11-22|  4| Emily| 65000|
|  41|        HR|1981-12-18|  5| David| 90000|
|  33|   Finance|1989-07-05|  6| Susan| 75000|
|  46|        IT|1976-03-15|  7|  Mike| 95000|
|  30|        IT|1992-05-12|  1|  John| 70000|
|  25|        HR|1997-02-28|  2| Alice| 60000|
|null|        IT|      null|  3|   Bob| 80000|
|  28|   Finance|1994-11-22|  4| Emily| 65000|
|  41|        HR|1981-12-18|  5| David| 90000|
|  33|   Finance|1989-07-05|  6| Susan| 75000|
|  46|        IT|1976-03-15|  7|  Mike| 95000|
|  30|   Finance|1992-06-30| 10|Sophie| 62000|
|  30|   Finance|1992-06-30| 10|Sophie| 62000|
|  25|   Finance|1997-02-28|  2| Alice| 90000|
|  25|   Finance|1997-02-28|  2| Alice| 90000|
|  28|   Fina

In [0]:
df_json.printSchema()

root
 |-- age: long (nullable = true)
 |-- department: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)



In [0]:
df=df_csv.union(df_json)

df1=df.dropDuplicates()
df1.show()

+----+-------+----------+----+------+----------+
|  id|   name|       dob| age|salary|department|
+----+-------+----------+----+------+----------+
|   2|  Alice|1997-02-28|  25| 60000|        HR|
|   3|    Bob|      null|null| 80000|        IT|
|   4|  Emily|1994-11-22|  28| 65000|   Finance|
|   1|   John|1992-05-12|  30| 70000|        IT|
|null|     IT|      null|   3|   Bob|     80000|
|  33|Finance|1989-07-05|   6| Susan|     75000|
|  41|     HR|1981-12-18|   5| David|     90000|
|  25|     HR|1997-02-28|   2| Alice|     60000|
|  30|     IT|1992-05-12|   1|  John|     70000|
|  28|Finance|1994-11-22|   4| Emily|     65000|
|  46|     IT|1976-03-15|   7|  Mike|     95000|
|  30|Finance|1992-06-30|  10|Sophie|     62000|
|  25|Finance|1997-02-28|   2| Alice|     90000|
|  28|Finance|1994-11-22|   4| Emily|     70000|
|  39|     IT|1983-10-14|   9| James|     87000|
|  27|     HR|1995-08-20|   8|  Lisa|     58000|
+----+-------+----------+----+------+----------+



In [0]:
print(df_json.columns,df_csv.columns)

['age', 'department', 'dob', 'id', 'name', 'salary'] ['id', 'name', 'dob', 'age', 'salary', 'department']


In [0]:
df_json = df_json.select(df_csv.columns)
df.show()
print(df.count())

+----+-------+----------+----+------+----------+
|  id|   name|       dob| age|salary|department|
+----+-------+----------+----+------+----------+
|   1|   John|1992-05-12|  30| 70000|        IT|
|   2|  Alice|1997-02-28|  25| 60000|        HR|
|   3|    Bob|      null|null| 80000|        IT|
|   4|  Emily|1994-11-22|  28| 65000|   Finance|
|  30|     IT|1992-05-12|   1|  John|     70000|
|  25|     HR|1997-02-28|   2| Alice|     60000|
|null|     IT|      null|   3|   Bob|     80000|
|  28|Finance|1994-11-22|   4| Emily|     65000|
|  41|     HR|1981-12-18|   5| David|     90000|
|  33|Finance|1989-07-05|   6| Susan|     75000|
|  46|     IT|1976-03-15|   7|  Mike|     95000|
|  30|     IT|1992-05-12|   1|  John|     70000|
|  25|     HR|1997-02-28|   2| Alice|     60000|
|null|     IT|      null|   3|   Bob|     80000|
|  28|Finance|1994-11-22|   4| Emily|     65000|
|  41|     HR|1981-12-18|   5| David|     90000|
|  33|Finance|1989-07-05|   6| Susan|     75000|
|  46|     IT|1976-0

In [0]:
df.select(
    'salary',
    'age'
).show()
df.select(
    "name"

).show()
df.select(
    ['salary','age']
).show()
df.select(
    df.salary,
    df.age,
).show()

+------+----+
|salary| age|
+------+----+
| 70000|  30|
| 60000|  25|
| 80000|null|
| 65000|  28|
|  John|   1|
| Alice|   2|
|   Bob|   3|
| Emily|   4|
| David|   5|
| Susan|   6|
|  Mike|   7|
|  John|   1|
| Alice|   2|
|   Bob|   3|
| Emily|   4|
| David|   5|
| Susan|   6|
|  Mike|   7|
|Sophie|  10|
|Sophie|  10|
+------+----+
only showing top 20 rows

+-------+
|   name|
+-------+
|   John|
|  Alice|
|    Bob|
|  Emily|
|     IT|
|     HR|
|     IT|
|Finance|
|     HR|
|Finance|
|     IT|
|     IT|
|     HR|
|     IT|
|Finance|
|     HR|
|Finance|
|     IT|
|Finance|
|Finance|
+-------+
only showing top 20 rows

+------+----+
|salary| age|
+------+----+
| 70000|  30|
| 60000|  25|
| 80000|null|
| 65000|  28|
|  John|   1|
| Alice|   2|
|   Bob|   3|
| Emily|   4|
| David|   5|
| Susan|   6|
|  Mike|   7|
|  John|   1|
| Alice|   2|
|   Bob|   3|
| Emily|   4|
| David|   5|
| Susan|   6|
|  Mike|   7|
|Sophie|  10|
|Sophie|  10|
+------+----+
only showing top 20 rows

+------+--

In [0]:
#from pyspark.sql functions import *
#from pyspark.sql functions import col,min,max
from pyspark.sql import functions as f 


In [0]:
# df.select(
#     df.salary + .05*df.salary,
#     f.year(f.current_timestamp())-f.year("dob"),
#     f.year(f.current_timestamp())-f.year(f.col("dob")).alias("age")

# ).show()

df.select(
    (df.salary + .05*df.salary),
    f.expr("df.salary + .05*df.salary").alias("salaryRaised"),
    (
        f.year(f.current_timestamp())-f.year(f.col("dob"))
    ).alias("age")
).show

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1900797974329514>:8
      1 # df.select(
      2 #     df.salary + .05*df.salary,
      3 #     f.year(f.current_timestamp())-f.year("dob"),
      4 #     f.year(f.current_timestamp())-f.year(f.col("dob")).alias("age")
      5 
      6 # ).show()
----> 8 df.select(
      9     (df.salary + .05*df.salary),
     10     f.expr("df.salary + .05*df.salary").alias("salaryRaised"),
     11     (
     12         f.year(f.current_timestamp())-f.year(f.col("dob"))
     13     ).alias("age")
     14 ).show

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signatu

In [0]:
df=df.withColumn(
    "salary raised",
    (f.col("salary")+.1*f.col("salary"))

).show()

+----+-------+----------+----+------+----------+-------------+
|  id|   name|       dob| age|salary|department|salary raised|
+----+-------+----------+----+------+----------+-------------+
|   1|   John|1992-05-12|  30| 70000|        IT|      77000.0|
|   2|  Alice|1997-02-28|  25| 60000|        HR|      66000.0|
|   3|    Bob|      null|null| 80000|        IT|      88000.0|
|   4|  Emily|1994-11-22|  28| 65000|   Finance|      71500.0|
|  30|     IT|1992-05-12|   1|  John|     70000|         null|
|  25|     HR|1997-02-28|   2| Alice|     60000|         null|
|null|     IT|      null|   3|   Bob|     80000|         null|
|  28|Finance|1994-11-22|   4| Emily|     65000|         null|
|  41|     HR|1981-12-18|   5| David|     90000|         null|
|  33|Finance|1989-07-05|   6| Susan|     75000|         null|
|  46|     IT|1976-03-15|   7|  Mike|     95000|         null|
|  30|     IT|1992-05-12|   1|  John|     70000|         null|
|  25|     HR|1997-02-28|   2| Alice|     60000|       

In [0]:
df=df.withColumns(
    {
        "salary raised":(f.col("salary")+.1*f.col("salary"),
        "age": f.year(f.current_timestamp()) - f.year(f.col("dob")) 
    }
)

  File <command-1900797974329516>:4
    "age": f.year(f.current_timestamp()) - f.year(f.col("dob"))
         ^
SyntaxError: invalid syntax


In [0]:
df= df.withColumn(
    "age group"
    f.when(
        f.col("age")<=20,
        "Upto 20"
    ).when(
        (f.col("age")> 20) & (f.col("age")<=30) ,
        "21 to 30"

    ).when(
      (f.col("age")> 30) & (f.col("age")<=40) ,
        "31 to 40"  
    ).otherwise(
        "more than 40"
    )
)

In [0]:
df=df.drop("company","has_dob")
df=df.withColumnRenamed("name","first_name")

In [0]:
ref_df = spark.read.format("parquet").load("dbfs:/FileStore/tables/parquet")
ref_df.show()

In [0]:
df.join(
    ref_df,
    "department",
    "left",

).show()


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-3130358376213347>:1
----> 1 df.join(
      2     ref_df,
      3     "department",
      4     "left",
      5 
      6 ).show()

AttributeError: 'NoneType' object has no attribute 'join'

In [0]:
# df = df.join(
#     ref_df,
#     df.department == ref_df.department,
# ).show()

In [0]:
df.write.mode("overwrite").format("csv").save("dbfs:/FileStore/tables/newfinal")

In [0]:
%fs ls dbfs:/FileStore/tables/


In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("newfinal")

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-3884137933029565>:1
----> 1 df.write.mode("overwrite").format("delta").saveAsTable("newfinal")

AttributeError: 'NoneType' object has no attribute 'write'